In [6]:
import imports
import numpy as np
from sklearn.cluster import KMeans
import sklearn.datasets as skdatasets
#import sklearn.metrics as skmetrics 
#from metrics import accuracy
from initialisations import onoda2012 as onoda
from sklearn.decomposition import FastICA, PCA

In [7]:
dataset = skdatasets.load_iris()
data = dataset.data
target = dataset.target

K = 3

### TODO:

 * whiten
 * centre
 * standardise?

In [20]:
# Compute ICA
ica = FastICA(n_components=K) # return 3x4
#ica = FastICA() # returns 4x4 - completely different numbers
S_ = ica.fit_transform(data)  # Reconstruct signals
#A_ = ica.mixing_  # Get estimated mixing matrix

print(ica.components_) # The unmixing matrix

print(len(S_))
#print(A_)

# We can `prove` that the ICA model applies by reverting the unmixing.
#assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

[[-0.00622425 -0.19384275  0.01077996 -0.07276474]
 [ 0.20158736 -0.08864943 -0.02713314 -0.1392689 ]
 [-0.02289128 -0.00049625  0.04052786  0.03517525]]
150


In [10]:
# For comparison, compute PCA
pca = PCA(n_components=K)
H = pca.fit_transform(data)  # Reconstruct signals based on orthogonal components
print(H)

[[-2.68412563  0.31939725 -0.02791483]
 [-2.71414169 -0.17700123 -0.21046427]
 [-2.88899057 -0.14494943  0.01790026]
 [-2.74534286 -0.31829898  0.03155937]
 [-2.72871654  0.32675451  0.09007924]
 [-2.28085963  0.74133045  0.16867766]
 [-2.82053775 -0.08946138  0.25789216]
 [-2.62614497  0.16338496 -0.02187932]
 [-2.88638273 -0.57831175  0.02075957]
 [-2.6727558  -0.11377425 -0.19763272]
 [-2.50694709  0.6450689  -0.07531801]
 [-2.61275523  0.01472994  0.10215026]
 [-2.78610927 -0.235112   -0.20684443]
 [-3.22380374 -0.51139459  0.06129967]
 [-2.64475039  1.17876464 -0.15162752]
 [-2.38603903  1.33806233  0.2777769 ]
 [-2.62352788  0.81067951  0.13818323]
 [-2.64829671  0.31184914  0.02666832]
 [-2.19982032  0.87283904 -0.12030552]
 [-2.5879864   0.51356031  0.21366517]
 [-2.31025622  0.39134594 -0.23944404]
 [-2.54370523  0.43299606  0.20845723]
 [-3.21593942  0.13346807  0.29239675]
 [-2.30273318  0.09870885  0.03912326]
 [-2.35575405 -0.03728186  0.12502108]
 [-2.50666891 -0.14601688